<a href="https://colab.research.google.com/github/kyletran77/Coursera_Capstone/blob/master/week%203%20part%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1 . Import the libraries to scrape the website**

In [0]:
import pandas as pd   #import the libraries we need
from bs4 import BeautifulSoup  #lib for table html
import requests
import numpy as np

**2. Use the wiki url and parse data into BeautifulSoup**

In [0]:
url  = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html5lib') #get wiki

In [0]:
postallist = [] #list for each cell
boroughlist = []
neighbourhood = []

**3.  Locate the rows from table and update each row into the lists.**

In [0]:
soup.find('table').find_all('tr')#table
soup.find('table').find_all('td')#rows
for row in soup.find('table').find_all('tr'):
  cells = row.find_all('td')#find all rows in table
  if(len(cells)>0):
    postallist.append(cells[0].text)
    boroughlist.append(cells[1].text)
    neighbourhood.append(cells[2].text.rstrip('\n'))# strip the ending to prevent newlines

**4. Create a dataframe**

In [0]:
tor_df = pd.DataFrame({'PostalCode': postallist,
                      'Borough':boroughlist,
                      'Neighborhood':neighbourhood}) #create the dataframe 
tor_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [0]:
tor_df1 = tor_df[tor_df['Borough'] != 'Not assigned'].reset_index(drop=True)
tor_df1.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [0]:
tor_group=tor_df1.groupby(['PostalCode', 'Borough'], as_index = False).agg(lambda x: ','.join(x))
tor_group.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


**5. Organize table to match question’s values**

In [0]:
columns_ = ['PostalCode','Borough','Neighborhood']
test_d = pd.DataFrame(columns= columns_)#create test dataframe

test_list = ['M5G','M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R','M9V','M9L','M5V','M1B', 'M5A']

for code in test_list:#update test dataframe where it takes each value in the test list to get the value
  test_d = test_d.append(tor_group[tor_group['PostalCode']==code], ignore_index = True)
test_d



,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens,Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale,Wexford"
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf..."


In [0]:
tor_group.shape

(103, 3)

**6. Create a dataframe with lat/long to see the location**

In [0]:
!wget -O Geospatial_Coordinates.csv 'https://cocl.us/Geospatial_data' #download the csv from coursera

--2019-08-14 15:36:43--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-08-14 15:36:44--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.24.197, 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-08-14 15:36:44--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [followin

In [0]:
location = pd.read_csv("Geospatial_Coordinates.csv")
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [0]:
location.rename(columns={'Postal Code':'PostalCode'}, inplace = True)
location.head()#change the name of the postal code to match other dataframe to be merged

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [0]:
tor_location = tor_group.merge(location, on = 'PostalCode', how = 'left')#combine the tor_group and location
tor_location.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [0]:
columnname = ['PostalCode','Borough', 'Neighborhood', 'Latitude', 'Longitude']#test to verify same as question
test_list = ['M5G','M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R','M9V','M9L','M5V','M1B', 'M5A']

test_dd = pd.DataFrame(columns=columnname) # test data

for code in test_list:#loop to use test list to update test_dd where the postal code is the test list
  test_dd = test_dd.append(tor_location[tor_location['PostalCode']==code], ignore_index = True)
test_dd

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale,Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420


**7. Assign the id and secret with a specific number for FourSquare**

In [0]:
client_id = 'EDLNTKPZ3EK0BQCNPOENJWUZ21U14QFXLE35CW1E5PMAXBDR' #FourSquare Information
secret = 'PM2RB3RBOHGYKVYB4EJGKRFXVBGE4SGJSJX4IORGQ5TLJEN1'
version = '20180605'
LIMIT='100'

In [0]:
from geopy.geocoders import Nominatim #import lib to find locations
import folium 

In [0]:
address = 'Toronto'# address for location

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)#locate geocode for address
latitude = location.latitude
longitude = location.longitude

In [0]:
map_tor = folium.Map(location =[latitude, longitude], zoom_start = 12)#folium map using lat, long 

for lat, lng, borough, neighborhood in zip(tor_location['Latitude'], tor_location['Longitude'], tor_location['Borough'], tor_location['Neighborhood']):
  label = '{}', '{}'.format(neighborhood, borough)# unzip each of the columns to format folii=um
  label = folium.Popup(label, parse_html = True)#
  folium.CircleMarker(
        [lat, lng],
        radius = 4,
        popup = label,
        color = 'red',
        fill =True,
        fill_color = '#3186cc',
        fill_opacity = 0.5).add_to(map_tor)#create the labeling using a red, late with a gray center
map_tor

**8. Create a special list that only contains Toronto**

In [0]:
borough_= list(tor_location.Borough.unique())#uni
tor_borough = []#create an empty list for borough that contains Toronto
for x in borough_: 
  if 'Toronto' in x:
    tor_borough.append(x)
tor_borough


['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [0]:
import json #libraries that is needed for the json files
from pandas.io.json import json_normalize

In [0]:
tor_location = tor_location[tor_location['Borough'].isin(tor_borough).reset_index(drop=True)]
tor_location.head()
tor_location.shape #location where Borough is updatated with the toronto only list

(38, 5)

**9. Create a function that gets the venue information**

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500): #create a function where the input holds the values below
    
    venues_list=[]#new list
    for name, lat, lng in zip(names, latitudes, longitudes):#prints out the names of the neighborhoods
        print(name)
            
        #use the foursquare api with the information needed
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            secret, 
            version, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items'] #use the request library to get the json file from the foursquare api
        
        # this wil give us the values that is needed for us 
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])#create a dataframe that contains the columns listed below 
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
tor_venues = getNearbyVenues(names = tor_location['Neighborhood'],  #input for the function with the neighborhoods
                         latitudes = tor_location['Latitude'],
                         longitudes = tor_location['Longitude'],
                         )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [0]:
print(tor_venues.shape)
tor_venues.head()

(1689, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


**10. Group Neighborhoods with one hot encoding**

In [0]:
tor_venues.groupby('Neighborhood').count() #group each neighborhood with the counts of each

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton,Exhibition Place,Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",46,46,46,46,46,46
Central Bay Street,82,82,82,82,82,82
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [0]:
toronto_hot = pd.get_dummies(tor_venues['Venue Category'], prefix = "", prefix_sep='') #create a dataframe using one hot encoding for each category
toronto_hot['Neighborhood'] = tor_venues['Neighborhood'] 

columng = [toronto_hot.columns[-1]]+list(toronto_hot.columns[:-1])
toronto_hot = toronto_hot[columng]
toronto_hot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,...,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
tor_group = toronto_hot.groupby('Neighborhood').mean().reset_index() #groupby neighborhood using that as the index
tor_group

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,...,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,0.000000,0.00,0.010000,0.000000,0.030000,0.0000,0.0000,0.000000,0.0000,0.000000,0.020000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.020000,0.000000,0.01,0.030000,0.000000,0.00000,0.01,0.020000,0.010000,...,0.000000,0.000000,0.0000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000,0.040000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.010000,0.000000,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.0000,0.0000,0.017857,0.0000,0.017857,0.035714,0.000000,0.000000,0.00,0.017857,0.017857,0.00,0.035714,0.000000,0.017857,0.0000,0.000000,0.000000,0.00,0.017857,0.000000,0.00000,0.00,0.000000,0.000000,...,0.000000,0.017857,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.017857,0.000000,0.00,0.000000,0.017857,0.017857,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.043478,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.043478,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.086957,0.000000,0.00000,0.00,0.000000,0.043478,...,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.043478,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.0625,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.062500,0.00000,0.00,0.000000,0.062500,...,0.000000,0.000000,0.0625,0.00,0.062500,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.000000,0.00000

**11. Use KMeans to cluster the neighborhoods**

In [0]:
from sklearn.cluster import KMeans #import libraries for kmeans
k=5 #number of clusters
torontocluster = tor_group.drop('Neighborhood',1) 
kmeans = KMeans(n_clusters=k, random_state=0).fit(torontocluster)#number of clusters using the kmeans method to fit
kmeans.labels_[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       4, 0, 1, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
merged = tor_location
merged['Cluster Labels']= kmeans.labels_#put the clusterlabels on the dataframe 
merged = merged.merge(tor_location.set_index('Neighborhood'), on='Neighborhood', how='left')#combine the two dataframes to get the long, lat, and other relevent data needed
merged.head(10)


,PostalCode_x,Borough_x,Neighborhood,Latitude_x,Longitude_x,Cluster Labels_x,PostalCode_y,Borough_y,Latitude_y,Longitude_y,Cluster Labels_y
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,M4E,East Toronto,43.676357,-79.293031,0
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,M4K,East Toronto,43.679557,-79.352188,0
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,M4L,East Toronto,43.668999,-79.315572,0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,M4M,East Toronto,43.659526,-79.340923,0
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,M4N,Central Toronto,43.728020,-79.388790,0
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,M4P,Central Toronto,43.712751,-79.390197,0
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,M4R,Central Toronto,43.715383,-79.405678,0
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,M4S,Central Toronto,43.704324,-79.388790,0
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,0,M4T,Central Toronto,43.689574,-79.383160,0
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,0,M4V,Central Toronto,43.686412,-79.400049,0


**12. Create a folium map of the clusters**

In [0]:

# use folium to create a new map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# randomize the colors for each of the clusters
cluster_color = {}
for cluster in range(k):
    cluster_color[cluster]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

# markers for each cluster using the location provided from the merged dataframe
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude_y'], 
                                  merged['Longitude_y'], 
                                  merged['Neighborhood'], 
                                  merged['Cluster Labels_x']):
    label_text = 'Cluster ' + str(cluster) + ' / ' + str(poi)
    label = folium.Popup(label_text, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        tooltip = label_text,
        radius=6,
        popup=label,
        color=cluster_color[cluster],
        fill=True,
        fill_color=cluster_color[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters